2장에서는 판다스를 사용해서 **csv**파일을 로드 해왔고, 사이킷런의 여러가지 변환기를 사용해서 전처리를 수행했었다. 예를 들어서 SimpleImputer, StandardScaler, OneHotIncoder

편리하긴 했지만, 대용량 데이터셋에서 텐서플로 모델을 훈련할 때는 텐서플로 자체의 데이터 로드 및 전처리 API인 `tf.data`를 사용하는게 좋다.

매우 효율적으로 데이터를 로드하고 전처리 할 수 있다. 멀티쓰레드와 큐를 사용해서 여러 파일에서 동시에 읽고, 샘플을 셔플링 하거나 배치로 만드는 등의 작업을 할 수 있다.

또 GPU가 현재 배치로 열심히 모델을 훈련시키는 동안 CPU는 다음 배치를 로드하고 전처리 하고 있을 수 있다.

그리고 메모리보다 큰 데이터셋을 처리할 수 있고, 하드웨어 리소스를 최대한 활용할 수 있어서 훈련 속도가 향상된다. 기본 기능으로 csv파일, 고정 길이의 이진 레코드를 가진 이진 파일, 텐서플로의 TFRecord 포맷을 사용하는 이진파일에서 데이터를 읽을 수 있다. 이 포맷은 길이가 다른 이진 레코드를 지원함.


케라스는 **모델에 포함**시킬 수 있는 강력하면서도 사용하기 쉬운 전처리 층을 제공한다. 제품 환경에 배포할 때 다른 전처리 코드 추가 없이 원시 데이터를 직접 주입할 수 있음.

이 챕터에서는 먼저 tf.data API와 TFRecoed 포맷을 살펴본다. 그런 다음 케라스 전처리 층과 이를 tf.data API와 함꼐 사용하는 방법을 알아본다.

마지막으로 데이터를 로드하고 전처리하는데 유용한 몇 가지 라이브러리를 간략하게 살펴본다.

# 데이터 API

데이터 API의 중심에는 tf.data.Dataset 개념이 있다. 이는 데이터 항목의 시퀀스는 나타낸다.

tf.data.Data.from_tensor_slices()를 사용해서 간단한 텐서로 데이터셋을 생성해보겠다. **그러니까 텐서로 Dataset을 만들어주는 함수인 것이다.**

텐서를 받아서 첫 번째 차원을 따라서 X의 각 원소가 아이템으로 표현되는 Dataset을 만든다. 즉 아래 코드에서는 10개의 아이템을 가지는 것이다.

In [1]:
import tensorflow as tf
from numpy import dtype

/Users/barrett11357/coding/pycharm/handson_ml_--/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
X = tf.range(10) # 임의의 데이터 생성
print(X)

dataset = tf.data.Dataset.from_tensor_slices(X)
print(dataset) # 10개의 아이템을 가짐

# for 문으로 순회할 수 있다.
for x in dataset:
    print(x)

tf.Tensor([0 1 2 3 4 5 6 7 8 9], shape=(10,), dtype=int32)
<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


2025-02-09 03:38:20.302018: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-02-09 03:38:20.302310: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-09 03:38:20.302320: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
I0000 00:00:1739039900.302640 2553019 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1739039900.302906 2553019 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-02-09 03:38:20.399930: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Dataset에는 텐서 튜플, 이름/텐서 쌍의 딕셔너리, 심지어 중첨된 튜플과 딕셔너리도 포함될 수 있다.

`from_tensor_slices()`는 중접 구조를 슬라이싱할 때, **데이터셋의 튜플/딕셔너리 구조를 유지하면서 그 안에 포함된 텐서만 슬라이싱한다.**



In [3]:
# 이렇게 복잡하게 충접된 딕셔너리도 슬라이싱해서 Dataset으로 바꿀 수 있다.

X_nested = {"a":([1,2,3],[4,5,6]), "b":[7,8,9]} # 숫자가 모두 동일하게 3개가 아니게 되면 차원 에러가 발생한다.

dataset = tf.data.Dataset.from_tensor_slices(X_nested)
for x in dataset:
    print(x)

{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=4>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=7>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=5>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=8>}
{'a': (<tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=6>), 'b': <tf.Tensor: shape=(), dtype=int32, numpy=9>}


2025-02-09 03:38:20.467280: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 연쇄 변환

데이터셋이 준비되면 변환 메서드를 호출해서 여러 종류의 변환을 수행할 수 있다.
**각 메서드는 새로운 데이터셋을 반환**해서 변환 메서드들을 체이닝 할 수 있다.

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10)) # tf.range(10)로 생성된건 Dataset이 아니라 텐서라서 repeat(), batch()를 쓸 수 없다.
dataset = dataset.repeat(3).batch(7) # 3번 반복하고, 7개씩 그룹으로 묶어서 배치를 만든 것이다. 마지막 배치는 데이터가 2개 밖에 없음.
for x in dataset:
    print(x)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


`map()`을 호출해서 각 아이템들에 변형을 가할 수도 있다. 아래 코드는 모든 아이템에 2를 곱한 새로운 데이터셋을 만든다.

In [5]:
dataset = dataset.map(lambda x: x * 2)
for x in dataset:
    print(x)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


2025-02-09 03:38:20.557426: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


map()을 사용해서 데이터에 어떤 전처리 작업도 적용할 수 있다. **복잡한 계산을 포함하는 경우 여러 쓰레드로 나누어서 속도를 높이는 것이 좋다.**

이를 위해서 num_parallel_calls 매개변수에 실행한 스레드 개수나, tf.data.AUTOTUNE을 지정할 수 있다. map()에 전달하는 메서드는 텐서플로 함수로 변환할 수 있어야 한다. <- 자세한 내용은 챕터12에 나옴.

아래 코드를 실행해보면 여러 쓰레드를 사용해서 데이터를 병렬로 처리해서 아이템의 순서가 보장되지 않은 모습을 볼 수 있다. 파이썬에서 멀티쓰레드 다시 실습해보고, 자바에서도 다시 좀 해보면 좋을 듯.

In [6]:
dataset = dataset.map(lambda x: x * 2, num_parallel_calls=tf.data.AUTOTUNE)
for x in dataset:
    print(x)

tf.Tensor([ 0  4  8 12 16 20 24], shape=(7,), dtype=int32)
tf.Tensor([28 32 36  0  4  8 12], shape=(7,), dtype=int32)
tf.Tensor([16 20 24 28 32 36  0], shape=(7,), dtype=int32)
tf.Tensor([ 4  8 12 16 20 24 28], shape=(7,), dtype=int32)
tf.Tensor([32 36], shape=(2,), dtype=int32)


또 `filter()`메서드를 사용해서 데이터셋을 필터링할 수도 있다.

아래 코드는 아이템의 합이 25보다 큰 배치만 담은 데이터셋을 만든다.

In [7]:
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10)).repeat(3).batch(7)

dataset = dataset.filter(lambda x : tf.reduce_sum(x) > 25)

for x in dataset:
    print(x)

tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)


데이터셋에 있는 몇 개의 아이템만 보고 싶을 떄는 `take()`메서드를 사용한다.

In [8]:
for x in dataset.take(2): # 2개의 아이템, 즉 2개의 배치만 나온다.
    print(x)

tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)


## 데이터 셔플링

4장에서 배웠는데, 경사하강법은 훈련 세트에 있는 샘플이 **독립적**이고 **동일한 분포**일 때 최고의 성능을 발휘한다.

데이터를 섞는 간단한 방법은 `shuffle()`메서드를 사용하는 방법이다.
이 메서는 원본 데이터셋의 처음 아이템을 buffer_size개수만큼 추출해서 버퍼에 채운다. 그 다음 새로운 아이템이 요청되면 이 버퍼에서 랜덤하게 하나를 꺼내서 반환한다.
그리고 원본 데이터셋에서 새로운 아이템을 하나 꺼내서 버퍼를 채운다. 버퍼가 텅 빌 때까지 이 과정을 계속 반복하는 것이다.

이 메서드를 사용하려면 버퍼 크기를 지정해야 하는데, 원리만 봐도 알겠듯이 버퍼 크기를 충분히 크게 하는 것이 중요하다.

In [9]:
dataset = tf.data.Dataset.range(10).repeat(2)
dataset = dataset.shuffle(buffer_size=4, seed=42).batch(7)

for x in dataset:
    print(x)

tf.Tensor([1 4 2 3 5 0 6], shape=(7,), dtype=int64)
tf.Tensor([9 8 2 0 3 1 4], shape=(7,), dtype=int64)
tf.Tensor([5 7 9 6 7 8], shape=(6,), dtype=int64)


2025-02-09 03:38:20.670603: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## 메모리 용량보다 큰 대규모 데이터셋에는 어떻게 해야할까?

메모리 용량보다 큰 대규모 데이터셋의 경우 버퍼가 데이터셋에 비해 작을 수 밖에 없기 때문에 이 방법 만으로는 충분하지 않다.

이를 해결하는 방법은 원본 데이터 자체를 섞는 것이다. (리눅스에서는 shuffle())

이렇게 하면 셔플링 효과가 크게 향상

그런데 원본 데이터가 섞여 있어도 일반적으로 에포크마다 한 번씩 더 섞어야 한다.
그렇지 않으면 에포크마다 동일한 순서가 반복되어서 모델에 편향이 추가된다. 원본 데이터에 존재하는 가짜 패턴 때문이다.

더 섞기 위해서 사용하는 방법으로 원본 데이터를 여러 파일로 나눈 다음에 훈련하는 동안 램덤으로 몇 개를 골라서 읽는 것이다.

그런데 아직도 문제가 있는게, 동일한 파일에 있는 샘플들을 여전히 같은 순서로 읽힌다.

이를 피하기 위해서 파일 여러 개를 랜덤으로 선택하고, 파일에서 동시에 읽은 레코드(행, 튜플)를 돌아가면서 반환할 수 있다.
그리고 또 여기에 shuffle()메서드를 사용해서 그 위에 셔플링 버퍼를 추가할 수 있다.

이제 직접 해보자!!!




## 여러 파일에서 한 줄씩 번갈아 가면서 읽기

In [10]:
# 추가 코드 - 캘리포니아 주택 데이터셋을 가져오고, 분할하고, 정규화합니다.
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [11]:
# 추가 코드 - 데이터셋을 20개 파일로 분할하여 CSV 파일로 저장합니다.
import numpy as np
from pathlib import Path

def save_to_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = Path() / "datasets" / "housing"
    housing_dir.mkdir(parents=True, exist_ok=True)
    filename_format = "my_{}_{:02d}.csv"

    filepaths = []
    m = len(data)
    chunks = np.array_split(np.arange(m), n_parts)
    for file_idx, row_indices in enumerate(chunks):
        part_csv = housing_dir / filename_format.format(name_prefix, file_idx)
        filepaths.append(str(part_csv))
        with open(part_csv, "w") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(",".join([repr(col) for col in data[row_idx]]))
                f.write("\n")
    return filepaths

train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)

# 분할된 csv 파일들의 경로는 담은 변수들
train_filepaths = save_to_csv_files(train_data, "train", header, n_parts=20)
valid_filepaths = save_to_csv_files(valid_data, "valid", header, n_parts=10)
test_filepaths = save_to_csv_files(test_data, "test", header, n_parts=10)

# 꼭 경로들을 하나하나 리스트에 담지 않고, 파일 패턴을 사용할 수도 있다고 한다.

In [12]:
print(train_filepaths)

['datasets/housing/my_train_00.csv', 'datasets/housing/my_train_01.csv', 'datasets/housing/my_train_02.csv', 'datasets/housing/my_train_03.csv', 'datasets/housing/my_train_04.csv', 'datasets/housing/my_train_05.csv', 'datasets/housing/my_train_06.csv', 'datasets/housing/my_train_07.csv', 'datasets/housing/my_train_08.csv', 'datasets/housing/my_train_09.csv', 'datasets/housing/my_train_10.csv', 'datasets/housing/my_train_11.csv', 'datasets/housing/my_train_12.csv', 'datasets/housing/my_train_13.csv', 'datasets/housing/my_train_14.csv', 'datasets/housing/my_train_15.csv', 'datasets/housing/my_train_16.csv', 'datasets/housing/my_train_17.csv', 'datasets/housing/my_train_18.csv', 'datasets/housing/my_train_19.csv']


In [13]:
# 파일 하나를 열어서 몇 줄만 읽어본다.
print("".join(open(train_filepaths[0]).readlines()[:4]))

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442
5.3275,5.0,6.490059642147117,0.9910536779324056,3464.0,3.4433399602385686,33.69,-117.39,1.687
3.1,29.0,7.5423728813559325,1.5915254237288134,1328.0,2.2508474576271187,38.44,-122.98,1.621



이제 이런 **파일 경로를 담은 데이터셋**을 만든다. `tf.data.Dataset.list_files()`가 파일 경로를 섞어서 반환한다.

In [14]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42) # 파일 경로를 담은 데이터셋을 반환한다. 파일 경로들을 섞어준다.
for x in filepath_dataset:
    print(x)

tf.Tensor(b'datasets/housing/my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_00.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_12.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_19.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_13.csv', shape=(), dtype=string)
tf.Tensor(b'datasets/housing/my_train_15.csv', shape=(), dtype=string)
tf.Ten

이제 `interleave()`메서드를 호출해서 한 번에 다섯 개의 파일을 한 줄씩 번갈아 읽는다.

이 메서드는 filepath_dataset에 있는 다섯 개의 파일 경로에서 데이터를 데이터를 읽는 데이터셋을 만든다.

전달한 함수를 각 파일에 대해 호출해서 새로운 데이터셋을 만드는 것이다.

1. **전체 구조**
   - `filepath_dataset`에서 파일 경로들을 가져와서
   - 각 파일을 `TextLineDataset`으로 변환하여 읽습니다
   - 5개의 파일을 동시에 번갈아가며 읽습니다

2. **interleave 함수의 역할**
   - 첫 번째 인자 (lambda 함수):
     * 각 파일 경로를 받아서
     * `TextLineDataset`을 생성
     * `skip(1)`로 첫 줄(헤더)를 건너뜀

3. **매개변수 설명**
   - `cycle_length=5`:
     * 5개의 파일을 동시에 열어서
     * 번갈아가며 한 줄씩 읽음
   - `num_parallel_calls=tf.data.AUTOTUNE`:
     * 멀티스레드로 파일 읽기 수행
     * 시스템이 자동으로 최적의 스레드 수 결정

4. **데이터 읽기 순서**
   ```
   파일1의 첫째줄 → 파일2의 첫째줄 → 파일3의 첫째줄 → 파일4의 첫째줄 → 파일5의 첫째줄
   파일1의 둘째줄 → 파일2의 둘째줄 → ... (반복)
   ```

In [15]:
n_readers = 5 # 동시에 5개의 파일을 읽겠다는 뜻
dataset  = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1), # 각 파일을 한 줄씩 읽는 함수. slip(1)은 열 이름은 건너뛰기 위해 넣음.
    cycle_length=n_readers,
    num_parallel_calls=tf.data.AUTOTUNE) # 멀티쓰레드로 수행

In [16]:
for x in dataset.take(5): # dataset.take(1)은 하나의 아이템만 포함하는 "새로운 데이터셋"을 반환하는 것이다. 그래서 이렇게 순회를 돌리는 것이다.
    print(x)

tf.Tensor(b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418', shape=(), dtype=string)
tf.Tensor(b'2.4792,24.0,3.4547038327526134,1.1341463414634145,2251.0,3.921602787456446,34.18,-118.38,2.0', shape=(), dtype=string)
tf.Tensor(b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67', shape=(), dtype=string)
tf.Tensor(b'2.1856,41.0,3.7189873417721517,1.0658227848101265,803.0,2.0329113924050635,32.76,-117.12,1.205', shape=(), dtype=string)
tf.Tensor(b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215', shape=(), dtype=string)


## 데이터 전처리

앞에서 만든 dataset은 각 샘플이 바이트 문자열이 담긴 텐서로 저장되어 있는 것이다.

이제 **문자열을 파싱**하고, 데이터 스케일을 조정하는 등 약간의 전처리가 필요하다.

이런 전처리를 수행하기 위해 사용자 정의 함수를 몇 개 만들어 본다.

In [17]:
# 추가 코드 - 각 특성의 평균 및 표준 편차 계산
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

StandardScaler와 동일한 기능을 하는 코드를 작성한다.

In [18]:
X_mean, X_std = scaler.mean_, scaler.scale_  # 각 특성의 평균과 표준편차를 담은 1D 텐서
n_inputs = 8 # 데이터의 특성 수

# csv 문자열 한 줄을 받아서 파싱을 해서 텐서로 반환하는 함수
def parse_csv_line(line) :
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)] # 파싱 할 때 누락된 값을 위한 기본값 설정
    fields = tf.io.decode_csv(line, record_defaults=defs)
    return tf.stack(fields[:-1]), tf.stack(fields[-1:]) # 데이터와 레이블을 각각 반환하는 것이다.

# 평균과 표준편자를 이용해서 각 샘플을 표준화 하는 함수. sklearn의 StandradScaler와 똑같은 기능을 한다.
def preprocess_csv_line(line):
    x, y = parse_csv_line(line)
    return (x - X_mean) / X_std, y

In [19]:
# 테스트해보면 잘 작동한다.
preprocess_csv_line(b'4.2083,44.0,5.323204419889502,0.9171270718232044,846.0,2.3370165745856353,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.16579159,  1.216324  , -0.05204396, -0.39210168, -0.5277444 ,
        -0.26334172,  0.8543046 , -1.3072058 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

## 데이터 적재와 전처리 합치기

좋아. 이제 앞에서 해본 것들을 전부 다 합친 하나의 헬퍼 함수를 만들어본다.

1. 파일 경로가 담긴 리스트가 들어오면, 이걸 다 섞으면서 파일 경로 데이터셋에 담는다.
2. 파일 경로 데이터 셋에서 n개의 파일 경로만 랜덤으로 뽑아서 번갈아가면서 한 줄씩 읽어와서 dataset에 저장한다.
3. dataset에 저장된 csv 문자열들을 파싱해서 숫자로 만들고, 표준화를 가한다.
4. 배치로 쪼개서 반환한다.

In [20]:
def csv_reader_dataset(filepaths, seed=42, n_readers=5, n_inputs = 8, buffer_size=10000, batch_size=32):
    filepath_dataset = tf.data.Dataset.list_files(filepaths, seed=seed)

    dataset = filepath_dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers,
        num_parallel_calls=tf.data.AUTOTUNE) # 멀티 스레드 사용

    dataset = dataset.map(preprocess_csv_line, num_parallel_calls=tf.data.AUTOTUNE) # 멀티스레드 사용

    dataset = dataset.shuffle(buffer_size=buffer_size, seed=seed)

    return dataset.batch(batch_size).prefetch(1)

마지막에 `prefetch(1)`을 호출하면 데이터셋은 항상 한 배치가 미리 준비되도록 최선을 다한다. 훈련 알고리즘이 한 배치로 작업을 하는 동안 이 데이터셋은 동시에 다음 배치를 준비한다.

매개변수로 `tf.data.AUTOTUNE을 전달하면 텐서플로가 자동으로 얼마 만큼의 배치를 준비해야 하는지 결정한다.

In [21]:
dataset = csv_reader_dataset(train_filepaths, n_readers=5, buffer_size=10000, batch_size=32) # 변환

# 한 배치의 데이터와 레이블을 각각 구경해본다.
for line in dataset.take(1):
    x,y  = line
    print(x)

for line in dataset.take(1):
    x,y  = line
    print(y)

tf.Tensor(
[[-1.39574516e+00 -4.94068526e-02 -2.28308082e-01  2.26482734e-01
   2.25936222e+00  3.52006316e-01  9.66738582e-01 -1.41216016e+00]
 [ 2.71126270e+00 -1.07781315e+00  6.94131434e-01 -1.48705527e-01
   5.18105030e-01  3.50729406e-01 -8.22851539e-01  8.06805968e-01]
 [-1.34846434e-01 -1.86889505e+00  1.03250686e-02 -1.37871787e-01
  -1.28934488e-01  3.14351842e-02  2.68705696e-01  1.32121444e-01]
 [ 9.03177410e-02  9.78999496e-01  1.32758200e-01 -1.37537822e-01
  -2.33884469e-01  1.02115452e-01  9.76108432e-01 -1.41216016e+00]
 [ 5.21880873e-02 -2.02711129e+00  2.94010907e-01 -2.40344461e-02
   1.62187666e-01 -2.84451842e-02  1.41179419e+00 -9.37379360e-01]
 [-6.72276020e-01  2.97013279e-02 -7.69225836e-01 -1.50867864e-01
   4.96202409e-01 -2.74199769e-02 -7.85372376e-01  7.71822453e-01]
 [-8.11177075e-01  3.46134037e-01 -2.18263835e-01 -8.01026970e-02
   6.63637593e-02  2.67242640e-01  1.93749100e-01  3.02040339e-01]
 [-6.89402997e-01  1.84918952e+00 -8.05119038e-01 -8.77811

## 케라스와 Dataset 사용하기

앞에서 만든 csv_reader_dataset() 을 사용해서 훈련, 검증, 테스트 데이터를 가공해보자.

In [22]:
train_set = csv_reader_dataset(train_filepaths, n_readers=5, buffer_size=10000, batch_size=32)
valid_set = csv_reader_dataset(valid_filepaths, n_readers=5, buffer_size=10000, batch_size=32)
test_set = csv_reader_dataset(test_filepaths, n_readers=5, buffer_size=10000, batch_size=32)

그리고 이제 얘네를 가지고 훈련할 수 있다.

그런데 앞에서 했던 것처럼 X_train, y_train이렇게 따로 따로 전달하는게 아니고, 현재 train_set, valid_set, test_set에는 다 레이블이 포함되어 있기 때문에 그냥 넣어주면 된다.

데이터셋은 매 에포크마다 셔플된다. fit()메서드가 에포크마다 랜덤한 순서로 훈련 데이터셋을 한 번씩 반복한다.

In [23]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="sgd")
model.fit(train_set, validation_data = valid_set, epochs = 1, verbose=2)

2025-02-09 03:38:22.039669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-02-09 03:38:23.108991: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 2454283735648041019
2025-02-09 03:38:23.109003: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 4070741231059566193
2025-02-09 03:38:23.109006: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1524956519418951581
2025-02-09 03:38:23.109011: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 10413961157246517750
2025-02-09 03:38:23.109013: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 5854288071903206488
2025-02-09 03:38:23.109017: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous r

363/363 - 2s - 4ms/step - loss: 1.1600 - val_loss: 16.2034


2025-02-09 03:38:23.447173: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
2025-02-09 03:38:23.447187: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9766402229857520293
2025-02-09 03:38:23.447195: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 14291216707504950871
2025-02-09 03:38:23.447201: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 15065178145089487566


이렇게 tf.data API를 사용해서 강력한 입력 파이프라인을 만드는 방법을 배웠다. 지금까지 사용한 csv 파일은 간단하고 편리해서 널리 통용되지만 효율적이지 않고, 대규모의 복잡한 (이미지, 오디오) 데이터 구조를 지원하지 못한다. 이 대신 TFRecord를 사용하는 방법을 알아본다.

훈련과정에서 데이터를 적재하고 전처리하는 데 병목이 생기는 경우 유용하다.

# TFRecord 포맷

일단 코드는 건너 뛰고, 간단하게 설명만 남겨놓겠다.

TFRecord는 직렬화된 **프로토콜 버퍼**를 담고 있다. 이건 확장성이 좋고 효율적인 이진 **포맷**이다.

책에서 프로토콜 버퍼를 직접 정의하는 코드가 간단하게 나오는데, 일반적으로 텐서플로에서 사용할 프로토콜 버퍼 정의는 이미 컴파일 되어서 텐서플로 안에 파이썬 클래스로 포함되어 있다고 한다.
그래서 그냥 프로토콜 버퍼 정의에 따라 생성된 파이썬 클래스를 다룰 줄만 알면 됨.

그리고 그 파이썬 클래스를 직렬화해서 TFRecord에 저장하고, 이걸 다시 파싱해서 읽어내는 방식이다. 프로토콜 버퍼 정의만 제공하면 tf.io.decode_proto()함수를 사용해서 어떤 프로토콜 버퍼도 파싱할 수 있다.

하지만 일반적으로 텐서플로가 제공하는 전용 파싱 연산을 제공하는 사전 정의된 프로토콜 버퍼를 대신 사용하는 것이 좋다.
주요 프로토콜 버퍼로는 Example, SequenceExample이 있다.

Example 객체는 하나의 Features객체를 가진다. Features객체는 특성 이름과 Feature객체를 매핑한 딕셔너리를 가지고 있다. Feature객체는 Bytelist, Floatlist, Int64List중 하나를 담고 있다.

SequenceExample은 하나의 Features객체와 하나의 FeatureLists객체를 가지고 있다. FeatureLists객체는 이름과 FeatureList객체로 매핑된 딕셔너리를 가지고 있다. 그리고 FeatureList는 하나 이상의 Feature객체를 가지고 있다.

# 케라스의 전처리

신경망에 사용할 데이터를 준비하려면 일반적으로 수치 특성 정규화, 범주형 특성이나 텍스트 인코딩, 이미지 자르기와 크기 조정 등의 작업이 필요하다.

케라스는 모델에 포함할 수 있는 다양한 전처리 층을 제공한다.

## Nomalization 층

이건 앞에서도 써봤지만 **입력 특성을 표준화할 수 있는** Nomalization층을 제공한다. 이 층을 만들 때 각 특성의 평균과 분산을 지정할 수도 있고, 아니면 fit()을 호출해서 모델을 훈련하기 전에 adapt()에 훈련 세트를 전달해서 특성의 평균과 분산을 계산할 수 있다.

### 전처리 불일치 문제를 해결하는 방법

In [24]:
norm_layer = tf.keras.layers.Normalization()

model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(1, activation="relu")
])

model.compile(loss="mse", optimizer="sgd")

# norm layer를 통과시키기 위해 특성만 뽑아냄. 현재 train_set에는 특성과 레이블이 같이 있는 상태다.
# train_set은 이미 정규화가 된 상태이지만, 예시를 위해 한 번 더 정규화 한다ㅋㅋ
X_train = train_set.unbatch().map(lambda x, y: x)
norm_layer.adapt(X_train)

model.fit(train_set, validation_data = valid_set, epochs=1)

    363/Unknown 1s 3ms/step - loss: 2.7217

2025-02-09 03:38:25.138502: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8228886785398399008
2025-02-09 03:38:25.138547: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17957140967288023351
2025-02-09 03:38:25.138555: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 3076765436023824323
2025-02-09 03:38:25.138560: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 15456812406770037240
2025-02-09 03:38:25.138562: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 18381312475554204736
2025-02-09 03:38:25.138565: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9752210912658541858
2025-02-09 03:38:25.138567: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv 

363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 2.7188 - val_loss: 0.5959


2025-02-09 03:38:25.473200: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9878487928683836563
2025-02-09 03:38:25.473214: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1632688311992821238
2025-02-09 03:38:25.473226: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8728971235411287552


모델에 Nomalization 전처리 층을 포함시켰기 때문에 정규화에 대해 신경 쓰지 않고 이 모델을 제품에 배포할 수 있다. 원시 데이터를 바로 주입해도 됨.

**전처리 불일치 위험을 완전히 제거**한다.

이렇게 모델에 전처리 층을 포함시키는 것은 좋은 생각이고 간단하지만, **훈련속도를 느리게 만든다**

전처리가 훈련되는 동안 즉시 적용되기 때문에 에포크마다 매번 수행된다.

이보다는 **전체 훈련 세트를 딱 한 번 전처리 하는 것이 낫다.**

Nomalization층을 **독립적**으로 사용할 수 있다. <- 이건 뭔 얘기인가 했는데 Nomalization층을 모델 밖으로 빼서, 모델에 넣기 전에 전처리를 한다는 것임. 이런면 또 전처리 불일치 위험이 발생함.

In [25]:
norm_layer = tf.keras.layers.Normalization()

adapt_ds = train_set.unbatch().map(lambda features, label: features) # 배치를 풀고 특성만 뽑아내서 정규화 층을 적응시킨다.
norm_layer.adapt(adapt_ds)

'''
아래 코드와 같이 케라스 전처리 층은 tf.data API와 함께 사용할 수도 있다.
map()메서드에 케라스 전처리 층을 적용할 수 있음.
'''
train_set_normalized = train_set.map(lambda features, label: (norm_layer(features), label)) # 특성만 정규화 층을 통과시킨다.
valid_set_normalized = valid_set.map(lambda features, label: (norm_layer(features), label))

그래서 이 문제를 해결하기 위해서 adapt()메서드를 호출한 Nomalization층과 훈련된 모델을 포함하는 새로운 모델을 만든다.

훈련할 때는 Nomalization층을 독립적으로 빼서 정규화를 딱 한 번만 진행하는 식으로 훈련속도를 높이고, 배포할 때 이미 훈련 데이터에 적응시킨 정규화 층을 집어넣어서 배포하는 것이다.

이제 훈련 속도도 빠르고 전처리 불일치 문제도 해결했다!!!!!

In [26]:
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(1, activation="relu")
])
model.compile(loss="mse", optimizer="sgd")
model.fit(train_set, validation_data=valid_set, epochs=1)

    363/Unknown 1s 3ms/step - loss: 2.4226

2025-02-09 03:38:27.178090: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8228886785398399008
2025-02-09 03:38:27.178104: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 17957140967288023351
2025-02-09 03:38:27.178107: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 3076765436023824323
2025-02-09 03:38:27.178110: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 15456812406770037240
2025-02-09 03:38:27.178112: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 18381312475554204736
2025-02-09 03:38:27.178114: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9752210912658541858
2025-02-09 03:38:27.178115: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv 

363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 2.4198 - val_loss: 0.5100


2025-02-09 03:38:27.663741: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 9878487928683836563
2025-02-09 03:38:27.663757: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1632688311992821238
2025-02-09 03:38:27.663760: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8728971235411287552


케라스 전처리 층이 제공하는 것보다 더 많은 기능이 필요하다면 언제든지 자신만의 케라스 층을 만들면 된다!

아래는 예시 코드

In [27]:
class MyNormalization(tf.keras.layers.Layer):
    def adapt(self, X):
        self.mean_ = np.mean(X, axis=0, keepdims=True)
        self.std_ = np.std(X, axis=0, keepdims=True)

    def call(self, inputs):
        eps = tf.keras.backend.epsilon()  # 0 나눗셈 방지
        return (inputs - self.mean_) / (self.std_ + eps)

## Discretization, CategoryEncoding, StringLookup, Hashing

일단 패스하고 임베딩으로 넘어감.

## 임베딩을 사용해서 범주형 특성 인코딩하기

임베딩은 범주나 어휘 사전의 단어와 같은 고차원 데이터의 밀집 표현이다. 50000개의 범주가 있다면 원-핫 인코딩은 50000차원의 희소벡터를 만든다. 반면 임베딩은 상대적으로 작은 밀집 벡터이다. **벡터의 차원 수는 튜닝해야 할 하이퍼파라미터다.**

딥러닝에서 임베딩을 일반적으로 랜덤하게 초기화되고, 다른 모델 파라미터와 함께 경사 하강법으로 훈련된다. (잘 이해가 안될 수 있는데, 얘를 들어서 Embedding 레이어 같은 경우 (단어 수, 차원 수) shape의 임베딩 행렬을 가진다. 그리고 hello라는 단어가 인덱스 1번으로 등록되어 있으면 가중치 행렬의 인덱스 1번 행 가중치를 꺼내주는 식으로 벡터로 변환하는 것이다. 이 가중치들이 처음에는 랜덤으로 초기화되어 있는데, loss가 계산되면서 이 임베딩 행렬이 학습되는 것이다. 혹은 Word2Vec처럼 자기지도학습으로 사전 학습된 임베딩 행렬을 가져와서 사용할 수도 있다.)

**임베딩을 훈련할 수 있기 때문에 훈련 도중에 점차 향상**된다. **비슷한 범주들은 경사하강법이 더 가깝게 만든다.**

범주가 **유용하게 표현되도록 임베딩이 훈련되는 경향**이 있다. 이를 **표현 학습**이라고 한다.

케라스는 임베딩 행렬을 감싼 Embedding 층을 제공한다. 이 행렬은 범주마다 하나의 행을, 임베딩 차원하다 하나의 열을 가진다.

기본적으로 랜덤하게 초기화된다. 범주ID를 임베딩으로 변환하기 위해 임베딩 행렬에서 범주에 해당하는 행을 찾아서 반환한다. 이게 전부다.

In [28]:
tf.random.set_seed=42

# 이 층은 아직 훈련되지 않았기 때문에 일언 인코딩 값은 그냥 랜덤한 숫자다.
embedding_layer = tf.keras.layers.Embedding(input_dim=5, output_dim=2)
embedding_layer(np.array([2,4,2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.03145789, -0.02335962],
       [-0.04922844, -0.03540355],
       [-0.03145789, -0.02335962]], dtype=float32)>

범주형 텍스트 특성을 임베딩하기 위해 `StringLookup`층과 `Embedding`층을 아래 코드와 같이 연결할 수 있다.

In [29]:
tf.random.set_seed=42

ocean_prox = ["<1H OCEAN","INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]

str_lookup_layer = tf.keras.layers.StringLookup()
str_lookup_layer.adapt(ocean_prox) # 어휘사전을 만든다.

# 범주형 텍스트 특성을 받아서 벡터로 임베딩하는 모델을 만든다.
lookup_and_embed = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=[], dtype=tf.string),
    str_lookup_layer,
    tf.keras.layers.Embedding(input_dim=str_lookup_layer.vocabulary_size(), output_dim=2)
    # 입력 차원은 어휘사전의 크기와 같아야 한다. 단어수+OOV 버킷 수(1개)
    # 이 코드에서는 2차원 임베딩을 사용하지만 일반적으로 임베딩은 10~300차원으로 구성된다고 한다. 훈련세트의 크기에 따라 다르므로 하이퍼파라미터로써 튜닝해야 한다.
])

lookup_and_embed(np.array(["<1H OCEAN","INLAND", "NEAR OCEAN"]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 0.03800198,  0.02812399],
       [ 0.00636127, -0.02606433],
       [-0.02713052, -0.03641682]], dtype=float32)>

이를 모두 연결해서 수치형 특성과 범주형 특성을 입력받아서 처리하는 모델을 만들 수 있다.

In [30]:
num_input = tf.keras.layers.Input(shape=[8], name="num")
cat_input = tf.keras.layers.Input(shape=[], name="cat", dtype=tf.string)

cat_encoded = lookup_and_embed(cat_input)

encoded_inputs = tf.keras.layers.concatenate([num_input, cat_encoded])
output = tf.keras.layers.Dense(1)(encoded_inputs)

model = tf.keras.Model(inputs=[num_input, cat_input], outputs=output)

model.compile(loss="mse", optimizer="sgd")

원-핫 인코딩 다음에 뒤따르는 활성화 함수가 없고, 편향이 없는 Dense층은 Embedding층과 동등한 역할을 한다고 한다.

## 텍스트 전처리

케라스는 기본적인 텍스트 전처리를 위한 TextVectorization 층을 제공한다. StringLookup 층과 매우 비슷하게 층을 만들 때 vocabulary 매개변수로 어휘 사전을 전달하거나, adapt() 메서드를 사용해서 훈련데이터로부터 어휘 사전을 학습할 수 있다.

In [31]:
train_data = ["To be", "!(to be)", "That's the question", "Be, be, be."]

text_vec_layer = tf.keras.layers.TextVectorization()
text_vec_layer.adapt(train_data)
print(text_vec_layer.get_vocabulary())
text_vec_layer(["Be good!", "Question: be or be?"])

['', '[UNK]', 'be', 'to', 'the', 'thats', 'question']


<tf.Tensor: shape=(2, 4), dtype=int64, numpy=
array([[2, 1, 0, 0],
       [6, 2, 1, 2]])>

매개변수 ouput_mode="tf_idf"로 설정하면 훈련 데이터에 자주 등장하는 단어의 가중치는 높이고, 드물게 등장하는 단어의 가중치를 높인다.

해보려고 했는데, 단어의 수가 너무 적다고 안됨...

In [32]:
text_vec_layer = tf.keras.layers.TextVectorization(output_mode="tf_idf")
text_vec_layer.adapt(train_data)
text_vec_layer(["Be good!", "Question: be or be?"])

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.96725637, 0.6931472 , 0.        , 0.        , 0.        ,
        0.        ],
       [0.96725637, 1.3862944 , 0.        , 0.        , 0.        ,
        1.0986123 ]], dtype=float32)>

이런 텍스트 인코딩 방법은 몇가지 중요한 제약사항이 있다.

공백으로 단어가 구분되는 언어에만 사용 가능. 동음이의어를 구별하지 못함. 단어의 관계에 대한 힌트를 모델에게 주지 못함. 단어의 순서가 사라진다.

다른 좋은 방법은 훨씬 고급의 텍스트 전처리 기능을 제공하는 텐서플로 텍스트 라이브러리를 사용하는 것이다. 예를 들어서 텍스트를 단어보다 작은 토큰으로 분할할 수 있는 부분 단어 토크나이저를 제공한다.

## 사전 훈련된 언어 모델 구성 요소 사용하기

이건 맛보기만 하자. 원시 텍스트를 입력 받아 50차원 문장 임베딩을 출력하는 모듈이다. 문자열을 입력 받아서 하나의 임베딩을 출력한다.

내부적으로 문자열을 파싱하고, 대규모 말뭉치에서 사전 훈련된 임베딩 행렬을 사용해 각 단어를 임베딩한다.
그리고 모든 단어 임베딩의 평균을 계산하면 그 결과가 문장 임베딩이다.

처음에 이상한 오류가 나면서 안됐었는데 껐다가 켜니까 되네ㅋㅋ

In [36]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/nnlm/TensorFlow2/en-dim50/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])
print(embeddings)

tf.Tensor(
[[ 0.16589954  0.0254965   0.1574857   0.17688066  0.02911299 -0.03092718
   0.19445257 -0.05709129 -0.08631689 -0.04391516  0.13032274  0.10905275
  -0.08515751  0.01056632 -0.17220995 -0.17925954  0.19556305  0.0802278
  -0.03247919 -0.49176937 -0.07767699 -0.03160921 -0.13952136  0.05959712
   0.06858718  0.22386682 -0.16653948  0.19412343 -0.05491862  0.10997339
  -0.15811177 -0.02576607 -0.07910853 -0.258499   -0.04206644 -0.20052543
   0.1705603  -0.15314153  0.0039225  -0.28694248  0.02468278  0.11069503
   0.03733957  0.01433943 -0.11048374  0.11931834 -0.11552787 -0.11110869
   0.02384969 -0.07074881]
 [ 0.1437864   0.08291595  0.10897306  0.04464385 -0.03630389 -0.12605834
   0.20263346  0.12862863 -0.07873426 -0.01195358  0.0020956  -0.03080653
  -0.08019945 -0.18797135 -0.11973457 -0.26926652  0.05157408 -0.15541205
  -0.12221853 -0.27182642  0.08750801 -0.05013347  0.03012378  0.2053423
   0.10000334  0.18292566 -0.18280756  0.0780353   0.10936535 -0.10147726
  